# Testes com [Dask](https://dask.org/) em Python

Python é uma linguagem que une diferentes ferramentas para a ciência de dados, Dask é uma biblioteca em python para auxiliar a análise de grandes volumes de dados.

A programação paralela significa dividir o problema em partes separadas e unir as partes no final do processamento, Dask possui a escalabilidade existente no ecossistema python.

Os dados são divididos em chunks que podem ser manipulados indenpendentemente em paralelo.

In [3]:
from dask.distributed import Client
client = Client(n_workers = 4)

In [ ]:
gsdg

In [ ]:
client.close()